This is stage 2 for developing a discrete MDP wrapper for the highway environment based on discretizedMDP.ipynb.

The purpose of this code is to implement multiprocessing and set up a separate discretized mdp python script in the codebase.

In [1]:
# Move up one directory level to import local instance of highway environment
%cd ..
%pwd

d:\Project Files\HoLab\Codes\Mine\HighwayEnv-TRI


'd:\\Project Files\\HoLab\\Codes\\Mine\\HighwayEnv-TRI'

Import necessary modules

In [2]:
from highway_env.envs.discreteMDP_wrapper import HighwayDiscreteMDP

import cProfile
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)    # Other options: INFO, WARNING, ERROR, CRITICAL

import copy

<frozen importlib._bootstrap>:488: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.


Set up MDP

In [3]:
config = {
        "observation": {
            "type": "Kinematics",
            "vehicles_count": 50,
            "features": ["presence", "x", "y", "vx", "vy", "heading"],
            "normalize": False,
            "absolute": True,
            "order": "sorted",
            "observe_intentions": False,
            "include_obstacles": True
            }
        }
# highway_mdp = HighwayDiscreteMDP('highway-v0', config=config, render_mode='rgb_array')
# highway_mdp = HighwayDiscreteMDP('highway-v0', config=config, render_mode=None)
highway_mdp = HighwayDiscreteMDP('highway-fast-v0', config=config, render_mode=None)
highway_mdp.populate_MDPtable(max_depth = 4)

INFO:root:Current Depth: 4 | Frontier: 56 | Visited: 14 | Transitions:65
INFO:root:Current Depth: 4 | Frontier: 63 | Visited: 17 | Transitions:75
INFO:root:Current Depth: 4 | Frontier: 66 | Visited: 19 | Transitions:80
INFO:root:Current Depth: 4 | Frontier: 73 | Visited: 22 | Transitions:90
INFO:root:Current Depth: 4 | Frontier: 72 | Visited: 23 | Transitions:90
INFO:root:Current Depth: 4 | Frontier: 79 | Visited: 26 | Transitions:100
INFO:root:Current Depth: 4 | Frontier: 90 | Visited: 30 | Transitions:115
INFO:root:Current Depth: 4 | Frontier: 89 | Visited: 31 | Transitions:115
INFO:root:Current Depth: 4 | Frontier: 92 | Visited: 33 | Transitions:120
INFO:root:Current Depth: 4 | Frontier: 91 | Visited: 34 | Transitions:120
INFO:root:Current Depth: 4 | Frontier: 90 | Visited: 35 | Transitions:120
INFO:root:Current Depth: 4 | Frontier: 89 | Visited: 35 | Transitions:120
INFO:root:Current Depth: 4 | Frontier: 88 | Visited: 36 | Transitions:120
INFO:root:Current Depth: 4 | Frontier: 87 |

In [4]:
matMDP = highway_mdp.get_MDPmatrices()
plan = highway_mdp.value_iteration()

287
dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218

In [5]:
plan

PlanningResult(state_values={frozendict.frozendict({'position': (199.39, 8.0), 'speed': (24.43, 0.0), 'heading': 0.0}): 0.0, frozendict.frozendict({'position': (196.99, 8.0), 'speed': (20.09, 0.0), 'heading': 0.0}): 0.0, frozendict.frozendict({'position': (206.12, 4.01), 'speed': (29.34, -0.03), 'heading': -0.001}): 0.0, frozendict.frozendict({'position': (203.73, 4.34), 'speed': (24.98, -1.08), 'heading': -0.043}): 0.0, frozendict.frozendict({'position': (203.35, 0.35), 'speed': (24.98, -1.12), 'heading': -0.045}): 0.0, frozendict.frozendict({'position': (211.1, 8.0), 'speed': (29.91, 0.0), 'heading': 0.0}): 0.0, frozendict.frozendict({'position': (181.44, 8.0), 'speed': (29.34, 0.0), 'heading': 0.0}): 0.9976877960112223, frozendict.frozendict({'position': (203.72, 4.01), 'speed': (25.0, -0.04), 'heading': -0.002}): 0.0, frozendict.frozendict({'position': (203.46, 7.67), 'speed': (24.98, 1.04), 'heading': 0.042}): 0.0, frozendict.frozendict({'position': (201.65, 8.0), 'speed': (20.66,

Print MDP Tables

In [5]:
print(highway_mdp.env.unwrapped.action_type.actions_indexes)

highway_mdp.currMDP.transition

{'LANE_LEFT': 0, 'IDLE': 1, 'LANE_RIGHT': 2, 'FASTER': 3, 'SLOWER': 4}


{(frozendict.frozendict({'position': (153.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}),
  0): {frozendict.frozendict({'position': (178.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}): 1},
 (frozendict.frozendict({'position': (153.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}),
  1): {frozendict.frozendict({'position': (178.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}): 1},
 (frozendict.frozendict({'position': (153.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}),
  2): {frozendict.frozendict({'position': (178.3, 3.66), 'speed': (24.98, 1.08), 'heading': 0.043}): 1},
 (frozendict.frozendict({'position': (153.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}),
  3): {frozendict.frozendict({'position': (181.02, 0.0), 'speed': (29.34, 0.0), 'heading': 0.0}): 1},
 (frozendict.frozendict({'position': (153.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}),
  4): {frozendict.frozendict({'position': (176.23, 0.0), 'speed': (20.66, 0.0), 'heading': 0.0}): 1},
 (frozendict.frozendict({'pos

In [6]:
highway_mdp.currMDP.reward

{(frozendict.frozendict({'position': (153.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}),
  0,
  frozendict.frozendict({'position': (178.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0})): 0.7999999999999999,
 (frozendict.frozendict({'position': (153.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}),
  1,
  frozendict.frozendict({'position': (178.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0})): 0.7999999999999999,
 (frozendict.frozendict({'position': (153.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}),
  2,
  frozendict.frozendict({'position': (178.3, 3.66), 'speed': (24.98, 1.08), 'heading': 0.043})): 0.8327113850086175,
 (frozendict.frozendict({'position': (153.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}),
  3,
  frozendict.frozendict({'position': (181.02, 0.0), 'speed': (29.34, 0.0), 'heading': 0.0})): 0.9157750342935529,
 (frozendict.frozendict({'position': (153.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}),
  4,
  frozendict.frozendict({'position': (176.23, 0.0), 'speed

In [7]:
highway_mdp.currMDP.absorption

{frozendict.frozendict({'position': (178.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}): False,
 frozendict.frozendict({'position': (178.3, 3.66), 'speed': (24.98, 1.08), 'heading': 0.043}): False,
 frozendict.frozendict({'position': (181.02, 0.0), 'speed': (29.34, 0.0), 'heading': 0.0}): False,
 frozendict.frozendict({'position': (176.23, 0.0), 'speed': (20.66, 0.0), 'heading': 0.0}): False,
 frozendict.frozendict({'position': (196.57, 0.0), 'speed': (20.09, 0.0), 'heading': 0.0}): False,
 frozendict.frozendict({'position': (196.19, 3.6), 'speed': (20.05, 1.16), 'heading': 0.058}): False,
 frozendict.frozendict({'position': (198.96, 0.0), 'speed': (24.43, 0.0), 'heading': 0.0}): False,
 frozendict.frozendict({'position': (203.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}): False,
 frozendict.frozendict({'position': (203.3, 3.66), 'speed': (24.98, 1.08), 'heading': 0.043}): False,
 frozendict.frozendict({'position': (206.02, 0.0), 'speed': (29.34, 0.0), 'heading': 0.0}): False,
 fr

Find optimal policy

# TEST CODE

In [ ]:
import functools
import multiprocessing as mp
process_count = (mp.cpu_count()-2)
from pathos.multiprocessing import ProcessingPool as Pool    # Can run multiprocessing in interactive shell


max_depth = 2   # The number of steps to plan ahead
actions = highway_mdp.actions

start_env = (highway_mdp.initial_state, 0, highway_mdp)
visited = set()
transitions = {}
rewards = {}
absorption = {}
frontier = {start_env}
env2close = set()   # Maintain list of environments to close
while frontier:
    state, depth, curr_MDPstate = frontier.pop()
    visited.add(state)
    env2close.add(curr_MDPstate)
    if depth < max_depth:
        with Pool(process_count) as pool:
            return_vals = pool.map(functools.partial(HighwayDiscreteMDP.exeuteAction, envMDP=curr_MDPstate), actions)
        for action, next_state, trans_prob, reward, done, truncated, info, updated_MDPstate in return_vals:
            logging.debug(str(state[0]) + ' | ' + str(action) + ' | ' + str(next_state[0]))
            #2# Populate transitions table
            if (state[0], action) not in transitions:
                transitions[(state[0], action)] = {}
            if next_state[0] not in transitions[(state[0], action)]:
                transitions[(state[0], action)][next_state[0]] = trans_prob
            #2# Populate reward functions
            if (state[0], action) in rewards:
                rewards[(state[0], action, next_state[0])] += reward * trans_prob
            else:
                rewards[(state[0], action, next_state[0])] = reward * trans_prob
            #2# set absorption states
            absorption[next_state[0]] = done or truncated
            #2# Populate visited
            if next_state not in visited:
                frontier.add((next_state, depth + 1, updated_MDPstate))
            else:
                updated_MDPstate.env.close()
        MDPstatus = "Current Depth: " + str(depth) + " | Frontier: " + str(len(frontier)) +\
                    " | Visited: " + str(len(visited)) + " | Transitions:" + str(len(transitions))
        logging.info(MDPstatus)
env2close.discard(start_env[2]) #Keep the first step active
for curr_MDPstate in env2close:
    # close all duplicate environments
    curr_MDPstate.env.close()
env2close = set()   # Close ununsed environments at the end of the loop
# print(highway_mdp.actions)
highway_mdp.create_MDPTable(transition=transitions, absorption=absorption, reward=rewards, state_list=visited, action_list=actions)

INFO:root:Current Depth: 0 | Frontier: 5 | Visited: 1 | Transitions:5
INFO:root:Current Depth: 1 | Frontier: 9 | Visited: 2 | Transitions:10
INFO:root:Current Depth: 1 | Frontier: 13 | Visited: 3 | Transitions:15
INFO:root:Current Depth: 1 | Frontier: 16 | Visited: 5 | Transitions:20
INFO:root:Current Depth: 1 | Frontier: 13 | Visited: 12 | Transitions:25
INFO:root:Current Depth: 1 | Frontier: 4 | Visited: 17 | Transitions:25


In [ ]:
# |DEBUGGING CODE

print(highway_mdp.env.unwrapped.road.vehicles[0])
tmp = highway_mdp.env()
print(tmp.unwrapped.road.vehicles[0])
highway_mdp.step(1)
print(highway_mdp.env.unwrapped.road.vehicles[0])
print(tmp.unwrapped.road.vehicles[0])
highway_mdp.env = tmp
print(highway_mdp.env.unwrapped.road.vehicles[0])


# dir(highway_mdp.env.unwrapped)

next_state, reward, done, truncated, info = highway_mdp.step(1)
# next_state


MDPVehicle #544: [207.38302935   8.        ]
MDPVehicle #248: [207.38302935   8.        ]
1 [1.   1.   0.08 1.   0.   0.  ]
MDPVehicle #544: [232.38302935   8.        ]
MDPVehicle #248: [207.38302935   8.        ]
MDPVehicle #248: [207.38302935   8.        ]


In [ ]:
config = {
        "observation": {
            "type": "Kinematics",
            "vehicles_count": 50,
            "features": ["presence", "x", "y", "vx", "vy", "heading"],
            "normalize": False,
            "absolute": True,
            "order": "sorted",
            "observe_intentions": False,
            "include_obstacles": True
            }
        }

env = gym.make('highway-v0', config=config, render_mode='human')

In [75]:
vehicles = []
for veh_state in obs:
    feature_vals = {k: v for k,v in zip(config["observation"]["features"], veh_state)}
    veh = {}
    veh["position"] = tuple(np.round((feature_vals["x"],feature_vals["y"]), 2))
    veh["speed"] = tuple(np.round((feature_vals["vx"],feature_vals["vy"]), 2))
    veh["heading"] = np.round(feature_vals["heading"], 3)
    vehicles.append(frozendict(veh))
vehicles = tuple(vehicles)
vehicles

(frozendict.frozendict({'position': (1.0, 0.08), 'speed': (1.0, 0.0), 'heading': 0.0}),
 frozendict.frozendict({'position': (0.08, 0.0), 'speed': (-0.24, 0.0), 'heading': 0.0}),
 frozendict.frozendict({'position': (0.33, -0.04), 'speed': (-0.18, 0.0), 'heading': 0.003}),
 frozendict.frozendict({'position': (0.45, 0.04), 'speed': (-0.31, 0.0), 'heading': 0.005}),
 frozendict.frozendict({'position': (0.72, 0.0), 'speed': (-0.19, 0.0), 'heading': 0.0}),
 frozendict.frozendict({'position': (0.93, 0.04), 'speed': (-0.19, 0.0), 'heading': 0.0}),
 frozendict.frozendict({'position': (1.0, -0.08), 'speed': (-0.27, 0.0), 'heading': 0.0}),
 frozendict.frozendict({'position': (1.0, -0.04), 'speed': (-0.24, -0.0), 'heading': -0.0}),
 frozendict.frozendict({'position': (1.0, -0.0), 'speed': (-0.16, 0.0), 'heading': 0.001}),
 frozendict.frozendict({'position': (1.0, -0.08), 'speed': (-0.28, 0.0), 'heading': 0.0}),
 frozendict.frozendict({'position': (0.0, 0.0), 'speed': (0.0, 0.0), 'heading': 0.0}),


In [77]:
import sys

# sys.getsizeof(transitions)
sys.getsizeof(highway_mdp.copy_env())
# sys.getsizeof(int(10.0))


48